# Step 1. Define Your Task

In [2]:
# Define your specific task by defining validate func & postprocess func
from ptcompletion import OpenAITask

class LlamaPoemTask(OpenAITask):

    def validate(self, completion:str):
        '''
        Check if generated completion fits your intend format.
        Return: A bool value.
        '''
        return completion.startswith('LLAMA!')

    def postprocess(self, completion:str):
        '''
        Postprocess generated completion.
        Return: Postprocess result in a pytho dict.
        '''
        completion = completion.replace('LLAMA!', 'Oh my LLAMA!')
        return {'genrated_poem': completion.strip()}

# Create context & generation config

messages = [
    {
        'role': 'user',
        'content': 'Please write me two lines of poetry about llama. Your response should start with : LLAMA! ..'
    }
]

generation_config = dict(
    temperature=1.0,
    top_p=1,
    n=1, # n>1 is not supported in my default OpenAITask class. Please write your own task class if you want it.
    max_tokens = 128,
)

# OpenAI settings

api_key = r'YOUR_OPENAI_API_KEY_HERE'
model = 'gpt-3.5-turbo'

task = LlamaPoemTask(id=0, messages=messages, generation_config={}, model=model, api_key=api_key)

# Step 2. Test Your Task

In [3]:
while not task.completed:
    # Run until the task is completed.
    task.run()

In [4]:
print(task)

<class '__main__.LlamaPoemTask'>
Result: {'genrated_poem': 'Oh my LLAMA! With wool so soft and fine,\nGraceful and quirky, a wonder divine.'}
Completion: LLAMA! With wool so soft and fine,
Graceful and quirky, a wonder divine.
Input: [{'role': 'user', 'content': 'Please write me two lines of poetry about llama. Your response should start with : LLAMA! ..'}]


# 3. Run Your Tasks Parallelly with TaskQueue

In [5]:
messages_list = [[
    {
        'role': 'user',
        'content': f'Please write me two lines of poetry about llama and number {i}. Your response should start with : LLAMA! ..'
    }
] for i in range(10)]

tasks = [LlamaPoemTask(id = i,
                    messages=messages,
                    generation_config=generation_config,
                    model=model,
                    api_key=api_key)
                    for i, messages in enumerate(messages_list)]

In [6]:
from ptcompletion import TaskQueue

In [7]:
# See https://platform.openai.com/docs/guides/rate-limits for RPM limits.

tq = TaskQueue(requests_per_minute=60, max_rounds=3, max_requests_per_proc=16, log_file='tasks.log')

In [8]:
completed_tasks = tq.start(tasks)

[2023-05-15 10:04:59.951124] Task 0 Start.
[2023-05-15 10:05:00.953344] Task 1 Start.
[2023-05-15 10:05:01.955358] Task 2 Start.
Query Error:  APIConnectionError(message='Error communicating with OpenAI: ("Connection broken: InvalidChunkLength(got length b\'\', 0 bytes read)", InvalidChunkLength(got length b\'\', 0 bytes read))', http_status=None, request_id=None)
[2023-05-15 10:05:02.433979] Task 1 Failed in 1.481 seconds.
[2023-05-15 10:05:02.957453] Task 3 Start.
[2023-05-15 10:05:03.959630] Task 4 Start.
[2023-05-15 10:05:04.961729] Task 5 Start.
[2023-05-15 10:05:05.245120] Task 0 Completed in 5.294 seconds.
[2023-05-15 10:05:05.477898] Task 2 Completed in 3.523 seconds.
[2023-05-15 10:05:05.963858] Task 6 Start.
[2023-05-15 10:05:06.106010] Task 3 Completed in 3.149 seconds.
[2023-05-15 10:05:06.965758] Task 7 Start.
[2023-05-15 10:05:07.069804] Task 4 Completed in 3.11 seconds.
[2023-05-15 10:05:07.967709] Task 8 Start.
[2023-05-15 10:05:08.021300] Task 5 Completed in 3.06 secon

In [9]:
for idx, t in enumerate(completed_tasks):
    print(f'TASK Queue Idx {idx+1}, TASK {t.id}')
    print(t.result)

TASK Queue Idx 1, TASK 0
{'genrated_poem': 'Oh my LLAMA! Oh, how strange,\nAs a zero, you pose no range.'}
TASK Queue Idx 2, TASK 2
{'genrated_poem': "Oh my LLAMA! With two humps on their back,\nNumber 2, they're always on track."}
TASK Queue Idx 3, TASK 3
{'genrated_poem': 'Oh my LLAMA! Three toes on each foot,\nIn fields of green, they stand aloof.'}
TASK Queue Idx 4, TASK 4
{'genrated_poem': 'Oh my LLAMA! Four legs and fur,  \nfour times the charm, I concur.'}
TASK Queue Idx 5, TASK 5
{'genrated_poem': 'Oh my LLAMA! Oh graceful creature so alive,\nOh how five seems to be your lucky thrive.'}
TASK Queue Idx 6, TASK 6
{'genrated_poem': 'Oh my LLAMA! With six legs you prance so free,\nYour oddity brings smiles to me.'}
TASK Queue Idx 7, TASK 7
{'genrated_poem': 'Oh my LLAMA! Oh your beauty so divine,\nYour seven layers, I wish were mine.'}
TASK Queue Idx 8, TASK 8
{'genrated_poem': 'Oh my LLAMA! On the Andean plains so great, \nEight of them line up, just as fate.'}
TASK Queue Idx 9, T